# Model Fitting and Output Exploration

This notebook demonstrates how to:
1. Load data and fit the hierarchical Bayesian model
2. Examine convergence diagnostics
3. Extract and visualize player/team rankings
4. Generate match predictions
5. Perform posterior predictive checks

In [ ]:
import sys
from pathlib import Path

# Add parent directory to path for imports
sys.path.insert(0, str(Path.cwd().parent))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import arviz as az

from rugby_ranking.model.data import MatchDataset
from rugby_ranking.model.core import RugbyModel, ModelConfig
from rugby_ranking.model.inference import ModelFitter, InferenceConfig
from rugby_ranking.model.predictions import MatchPredictor

pd.set_option('display.max_columns', None)
sns.set_style('whitegrid')
az.style.use('arviz-darkgrid')

%config InlineBackend.figure_format = 'retina'

## 1. Load Data

In [ ]:
DATA_DIR = Path("../../Rugby-Data")  # Adjust path as needed

dataset = MatchDataset(DATA_DIR)
dataset.load_json_files()

df = dataset.to_dataframe(played_only=True)

# Filter to valid positions (1-23)
df = df[df['position'].between(1, 23)].copy()

print(f"Observations: {len(df):,}")
print(f"Players: {df['player_name'].nunique():,}")
print(f"Teams: {df['team'].nunique()}")
print(f"Matches: {df['match_id'].nunique():,}")

## 2. Build and Fit Model

We'll start with a single score type (tries) to validate the workflow.

In [ ]:
# Configure the model
config = ModelConfig(
    score_types=("tries",),  # Start with tries only
    player_effect_sd=0.5,
    team_effect_sd=0.3,
    position_effect_sd=0.5,
)

model = RugbyModel(config)
pymc_model = model.build(df, score_type="tries")

print("Model built successfully")
print(f"  Players: {len(model._player_ids)}")
print(f"  Team-seasons: {len(model._team_season_ids)}")

In [ ]:
# Fit using Variational Inference (fast)
inference_config = InferenceConfig(
    vi_n_iterations=30000,  # Reduce for faster iteration
    vi_method="advi",
)

fitter = ModelFitter(model, inference_config)

print("Fitting model with VI...")
trace = fitter.fit_vi(n_samples=2000)
print("Done!")

### Alternative: Full MCMC (slower but more accurate)

Uncomment to run full MCMC sampling. This takes longer but provides better uncertainty estimates.

In [ ]:
# # Full MCMC (uncomment to run)
# inference_config = InferenceConfig(
#     mcmc_draws=1000,
#     mcmc_tune=500,
#     mcmc_chains=4,
#     mcmc_cores=4,
# )
# fitter = ModelFitter(model, inference_config)
# trace = fitter.fit_mcmc()

## 3. Convergence Diagnostics

In [ ]:
# Summary statistics
diag = fitter.diagnostics()
print("Convergence Diagnostics:")
print(f"  R-hat max: {diag['r_hat_max']:.4f} (should be < 1.01)")
print(f"  R-hat mean: {diag['r_hat_mean']:.4f}")
print(f"  ESS bulk min: {diag['ess_bulk_min']:.0f} (should be > 400)")
print(f"  ESS tail min: {diag['ess_tail_min']:.0f}")
print(f"  Method: {diag['fit_method']}")

In [ ]:
# ArviZ summary for key parameters
summary = az.summary(trace, var_names=['alpha', 'eta_home', 'sigma_player', 'sigma_team'])
display(summary)

In [ ]:
# Posterior distributions for global parameters
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

az.plot_posterior(trace, var_names=['alpha'], ax=axes[0, 0])
axes[0, 0].set_title('Baseline Log-Rate (α)')

az.plot_posterior(trace, var_names=['eta_home'], ax=axes[0, 1])
axes[0, 1].set_title('Home Advantage (η)')

az.plot_posterior(trace, var_names=['sigma_player'], ax=axes[1, 0])
axes[1, 0].set_title('Player Effect SD (σ_player)')

az.plot_posterior(trace, var_names=['sigma_team'], ax=axes[1, 1])
axes[1, 1].set_title('Team Effect SD (σ_team)')

plt.tight_layout()
plt.show()

## 4. Player Rankings

In [ ]:
# Top try-scorers by estimated ability
player_rankings = model.get_player_rankings(top_n=30)
display(player_rankings)

In [ ]:
# Visualize top players with uncertainty
top_n = 20
rankings = model.get_player_rankings(top_n=top_n)

fig, ax = plt.subplots(figsize=(10, 8))

y_pos = np.arange(top_n)
ax.barh(y_pos, rankings['effect_mean'], xerr=rankings['effect_std'] * 1.96, 
        align='center', alpha=0.7, capsize=3)
ax.set_yticks(y_pos)
ax.set_yticklabels(rankings['player'])
ax.invert_yaxis()
ax.set_xlabel('Player Effect (log-rate)')
ax.set_title(f'Top {top_n} Try Scorers (with 95% CI)')
ax.axvline(x=0, color='gray', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

In [ ]:
# Player effect distribution
beta = trace.posterior['beta_player'].values.flatten()

fig, ax = plt.subplots(figsize=(10, 5))
ax.hist(beta, bins=50, density=True, alpha=0.7, edgecolor='black')
ax.set_xlabel('Player Effect')
ax.set_ylabel('Density')
ax.set_title('Distribution of Player Effects')
ax.axvline(x=0, color='red', linestyle='--', label='Average')
ax.legend()
plt.show()

print(f"Player effect range: [{beta.min():.2f}, {beta.max():.2f}]")
print(f"Player effect std: {beta.std():.3f}")

## 5. Team Rankings

In [ ]:
# Get most recent season
seasons = sorted(model._season_ids.keys())
current_season = seasons[-1] if seasons else None
print(f"Current season: {current_season}")

# Team rankings for current season
team_rankings = model.get_team_rankings(season=current_season, top_n=20)
display(team_rankings)

In [ ]:
# Visualize team rankings
if len(team_rankings) > 0:
    fig, ax = plt.subplots(figsize=(10, 8))
    
    y_pos = np.arange(len(team_rankings))
    colors = ['green' if x > 0 else 'red' for x in team_rankings['effect_mean']]
    
    ax.barh(y_pos, team_rankings['effect_mean'], xerr=team_rankings['effect_std'] * 1.96,
            align='center', alpha=0.7, capsize=3, color=colors)
    ax.set_yticks(y_pos)
    ax.set_yticklabels(team_rankings['team'])
    ax.invert_yaxis()
    ax.set_xlabel('Team Effect (log-rate)')
    ax.set_title(f'Team Rankings - {current_season} (with 95% CI)')
    ax.axvline(x=0, color='gray', linestyle='--', alpha=0.5)
    
    plt.tight_layout()
    plt.show()

## 6. Position Effects

In [ ]:
# Extract position effects
theta = trace.posterior['theta_position'].values
theta_mean = theta.mean(axis=(0, 1))
theta_std = theta.std(axis=(0, 1))

positions = {
    1: 'LH Prop', 2: 'Hooker', 3: 'TH Prop',
    4: 'Lock', 5: 'Lock', 6: 'Blindside', 7: 'Openside', 8: 'No.8',
    9: 'Scrum-half', 10: 'Fly-half', 11: 'Left Wing',
    12: 'Inside Ctr', 13: 'Outside Ctr', 14: 'Right Wing', 15: 'Fullback',
    16: 'Rep HK', 17: 'Rep LH', 18: 'Rep TH', 19: 'Rep LK', 20: 'Rep BR',
    21: 'Rep SH', 22: 'Rep FH', 23: 'Rep BK'
}

fig, ax = plt.subplots(figsize=(14, 6))

x_pos = np.arange(23)
colors = ['#1f77b4' if i < 8 else '#2ca02c' if i < 15 else '#7f7f7f' for i in range(23)]

ax.bar(x_pos, theta_mean, yerr=theta_std * 1.96, capsize=3, alpha=0.7, color=colors)
ax.set_xticks(x_pos)
ax.set_xticklabels([positions.get(i+1, str(i+1)) for i in range(23)], rotation=45, ha='right')
ax.set_ylabel('Position Effect (log-rate)')
ax.set_title('Try-Scoring Rate by Position (with 95% CI)')
ax.axhline(y=0, color='gray', linestyle='--', alpha=0.5)

# Add legend
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='#1f77b4', label='Forwards'),
    Patch(facecolor='#2ca02c', label='Backs'),
    Patch(facecolor='#7f7f7f', label='Replacements')
]
ax.legend(handles=legend_elements, loc='upper right')

plt.tight_layout()
plt.show()

## 7. Match Predictions

In [ ]:
# Create predictor
predictor = MatchPredictor(model, trace)

# Example prediction (teams only)
# Find some teams that exist in the current season
current_teams = [ts[0] for ts in model._team_season_ids.keys() if ts[1] == current_season]
print(f"Teams in {current_season}: {current_teams[:10]}...")

In [ ]:
# Predict a match (adjust team names as needed)
if len(current_teams) >= 2:
    home_team = current_teams[0]
    away_team = current_teams[1]
    
    try:
        prediction = predictor.predict_teams_only(
            home_team=home_team,
            away_team=away_team,
            season=current_season,
            n_samples=2000
        )
        
        print(prediction.summary())
        print()
        print(f"Detailed breakdown:")
        print(f"  Home ({home_team}): {prediction.home.mean:.1f} ± {prediction.home.std:.1f}")
        print(f"  Away ({away_team}): {prediction.away.mean:.1f} ± {prediction.away.std:.1f}")
        print(f"  90% CI Home: [{prediction.home.ci_lower:.0f}, {prediction.home.ci_upper:.0f}]")
        print(f"  90% CI Away: [{prediction.away.ci_lower:.0f}, {prediction.away.ci_upper:.0f}]")
    except ValueError as e:
        print(f"Could not predict: {e}")

In [ ]:
# Visualize score distributions
if 'prediction' in dir() and prediction is not None:
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    # Score distributions
    axes[0].hist(prediction.home.samples, bins=30, alpha=0.6, label=home_team, density=True)
    axes[0].hist(prediction.away.samples, bins=30, alpha=0.6, label=away_team, density=True)
    axes[0].set_xlabel('Predicted Score')
    axes[0].set_ylabel('Density')
    axes[0].set_title('Score Distributions')
    axes[0].legend()
    
    # Margin distribution
    margin = prediction.home.samples - prediction.away.samples
    axes[1].hist(margin, bins=40, alpha=0.7, color='purple', density=True)
    axes[1].axvline(x=0, color='red', linestyle='--', label='Draw')
    axes[1].axvline(x=margin.mean(), color='green', linestyle='-', label=f'Mean: {margin.mean():.1f}')
    axes[1].set_xlabel('Margin (Home - Away)')
    axes[1].set_ylabel('Density')
    axes[1].set_title('Predicted Margin')
    axes[1].legend()
    
    plt.tight_layout()
    plt.show()

## 8. Posterior Predictive Checks

Compare model predictions to observed data.

In [ ]:
# Compare observed vs predicted try rates by position
observed_rates = df.groupby('position').agg({
    'tries': 'sum',
    'minutes_played': 'sum'
})
observed_rates['rate_per_80'] = observed_rates['tries'] / observed_rates['minutes_played'] * 80

# Get model predictions for each position
alpha_mean = trace.posterior['alpha'].values.mean()
theta_mean = trace.posterior['theta_position'].values.mean(axis=(0, 1))

predicted_rate = np.exp(alpha_mean + theta_mean)

fig, ax = plt.subplots(figsize=(12, 6))

positions_list = list(range(1, 24))
x = np.arange(len(positions_list))
width = 0.35

obs_rates = [observed_rates.loc[p, 'rate_per_80'] if p in observed_rates.index else 0 
             for p in positions_list]
pred_rates = predicted_rate[:23]

ax.bar(x - width/2, obs_rates, width, label='Observed', alpha=0.7)
ax.bar(x + width/2, pred_rates, width, label='Predicted', alpha=0.7)

ax.set_xlabel('Position')
ax.set_ylabel('Tries per 80 minutes')
ax.set_title('Observed vs Predicted Try Rates by Position')
ax.set_xticks(x)
ax.set_xticklabels([positions.get(p, str(p)) for p in positions_list], rotation=45, ha='right')
ax.legend()

plt.tight_layout()
plt.show()

## 9. Save Checkpoint

Save the fitted model for later use.

In [ ]:
# Save checkpoint
checkpoint_path = fitter.save("tries_model_v1")
print(f"Saved to: {checkpoint_path}")

In [ ]:
# To load later:
# fitter = ModelFitter.load("tries_model_v1", model)

## 10. Exploring Individual Players

In [ ]:
# Look at a specific player's history and model estimate
def analyze_player(player_name):
    """Analyze a specific player's career and model estimate."""
    if player_name not in model._player_ids:
        print(f"Player '{player_name}' not found in model")
        return
    
    player_idx = model._player_ids[player_name]
    
    # Get player effect from posterior
    beta = trace.posterior['beta_player'].values[:, :, player_idx]
    
    # Get player's match history
    player_df = df[df['player_name'] == player_name].copy()
    
    print(f"=== {player_name} ===")
    print(f"Matches: {len(player_df)}")
    print(f"Total tries: {player_df['tries'].sum()}")
    print(f"Try rate: {player_df['tries'].sum() / player_df['minutes_played'].sum() * 80:.2f} per 80 min")
    print(f"Teams: {player_df['team'].unique().tolist()}")
    print(f"Seasons: {sorted(player_df['season'].unique().tolist())}")
    print()
    print(f"Model estimate:")
    print(f"  Effect mean: {beta.mean():.3f}")
    print(f"  Effect std: {beta.std():.3f}")
    print(f"  95% CI: [{np.percentile(beta, 2.5):.3f}, {np.percentile(beta, 97.5):.3f}]")
    
    # Plot posterior
    fig, ax = plt.subplots(figsize=(8, 4))
    ax.hist(beta.flatten(), bins=50, density=True, alpha=0.7)
    ax.axvline(x=0, color='gray', linestyle='--', label='League average')
    ax.axvline(x=beta.mean(), color='red', linestyle='-', label=f'Mean: {beta.mean():.3f}')
    ax.set_xlabel('Player Effect')
    ax.set_ylabel('Density')
    ax.set_title(f'Posterior Distribution: {player_name}')
    ax.legend()
    plt.show()

# Example: analyze top player
top_player = player_rankings.iloc[0]['player']
analyze_player(top_player)

In [ ]:
# Search for a specific player
def search_players(query):
    """Search for players by name."""
    matches = [p for p in model._player_ids.keys() if query.lower() in p.lower()]
    return sorted(matches)[:20]

# Example search
print("Players matching 'van der':", search_players("van der"))